# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770428128299                   -0.53    9.0    171ms
  2   -2.771686133425       -2.90       -1.31    1.0    114ms
  3   -2.771714328045       -4.55       -2.58    1.0    104ms
  4   -2.771714690875       -6.44       -3.82    1.0   94.3ms
  5   -2.771714714945       -7.62       -4.27    2.0    115ms
  6   -2.771714715228       -9.55       -5.20    1.0   97.5ms
  7   -2.771714715248      -10.69       -5.53    2.0    118ms
  8   -2.771714715250      -11.96       -5.99    1.0    108ms
  9   -2.771714715250      -13.07       -6.94    1.0    106ms
 10   -2.771714715250      -13.69       -7.18    2.0    124ms
 11   -2.771714715250   +  -14.35       -8.36    1.0    157ms


-0.00013457437699347993

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770518426556                   -0.52    9.0    246ms
  2   -2.771779819822       -2.90       -1.32    1.0    177ms
  3   -2.771801590846       -4.66       -2.44    1.0    114ms
  4   -2.771802003089       -6.38       -3.14    1.0   93.0ms
  5   -2.771802074296       -7.15       -4.38    2.0    134ms
  6   -2.771802074407       -9.95       -4.64    1.0   98.0ms
  7   -2.771802074473      -10.18       -5.38    1.0    100ms
  8   -2.771802074475      -11.70       -5.62    1.0    103ms
  9   -2.771802074476      -12.01       -6.48    1.0    103ms
 10   -2.771802074476      -13.37       -6.70    2.0    121ms
 11   -2.771802074476   +  -14.35       -7.08    1.0    108ms
 12   -2.771802074476   +  -15.35       -7.97    1.0    110ms
 13   -2.771802074476   +  -14.35       -8.70    1.0    108ms


0.017612222365186676

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457437699347993
Displaced dipole:  0.017612222365186676
Polarizability :   1.7746796742180155


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.28e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 2.36e-08
[ Info: GMRES linsolve in iteration 2; step 1: normres = 1.84e-09
[ Info: GMRES linsolve in iteration 2; step 2: normres = 3.55e-11
[ Info: GMRES linsolve in iteration 2; step 3: normres = 5.10e-12
┌ Info: GMRES linsolve converged at iteration 2, step 4:
│ * norm of residual = 6.40e-14
└ * number of operations = 13
Non-interacting polarizability: 1.9257125184676631
Interacting polarizability:     1.7736548415270859


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.